# Weighted Average

In [33]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Importing the dataset
dataset = pd.read_csv('CVD dataset2.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 16].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
17


In [34]:
dataNoCvd = data[~(data[:,16] > 0.0)]
dataCvd = data[~(data[:,16] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [35]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [36]:
def create_train_test_sets(cv,dataNoCvd,dataCvd):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)

    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)

        testSubset_NoCvd ,testSubset_Cvd ,testSubset_total = find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
     
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, 16, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, 16), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
    
        X_train.append(np.delete(trainSubset_total, 16, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, 16), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, train_total_NoCvd, train_total_Cvd

In [37]:
train_total, test_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9


In [38]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)

504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07142857142857142
ratio in test set: 0.08928571428571429


In [56]:
from xgboost import XGBClassifier


def my_fit(ratio,dataNoCvd,dataCvd,params):   
    Subarray=[]
    models = []
    # yarray
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0
    print("numOfSubsamples",numOfSubsamples)

    for i in range(numOfSubsamples):
        
        classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=params[5],
                  colsample_bynode=1, colsample_bytree=1, gamma=params[4],
                  learning_rate=params[0], max_delta_step=0, max_depth=params[1],
                  min_child_weight=params[3], missing=None, n_estimators=params[2], n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  reg_alpha=0, reg_lambda=params[6], scale_pos_weight=params[7], seed=None,
                  silent=None, subsample=1.0, verbosity=1)
    #     print(i)
    #Creating the training set for each model
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
#         print(Down)
#         print(Up)
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        print(len(Subarray))
        print(len(Subarray[i]))
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], 16, axis=1)
        y=np.delete(Subarray[i], slice(0, 16), axis=1)
        y=np.reshape(y, len(y))
        
        classifier.fit( X, y)
        models.append(classifier)
    return models   

In [58]:


def predict_weighted_average(models,X,weights):
#     print("efwef",len(weights), len(models))
    y_pred = []
    y_pred_models = []
#     weights = [0.16, 0.175, 0.2, 0.147, 0.148, 0.16]
#     weights = [0.15, 0.4, 0.2, 0.05, 0.05, 0.15]
#   weights = [0.16, 0.173, 0.2, 0.147, 0.16, 0.16]
    models_size = len(models)
#     print(models_size)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict_proba(X)[:,1]) 
        
    for j in range(X_size):
        sum = 0
        for i in range(models_size):
#             print(i)
#             print(y_pred_models[i][j])
#             print(weights[i])
            sum = sum + y_pred_models[i][j]*weights[i]
        avr = sum
#         /models_size
        if avr>= 0.5 :
            y_pred.append(1.)
        else:
            y_pred.append(0.)
    return y_pred        
  

In [41]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, train_total_NoCvd, train_total_Cvd,params, weights):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        


        models = my_fit( 1, train_total_NoCvd[i], train_total_Cvd[i],params)


        y_pred = predict_weighted_average(models,X_test,weights)

        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [12]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        
params = {
                    "learning_rate"    :[ 0.1, 0.2, 0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[1,2, 3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100, 200, 300],
                     "min_child_weight"       :[0.5,0.75, 1.0],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0, 0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[0.5,0.75, 1.0], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[1,2,3]
        }

In [19]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        
params = {
                    "learning_rate"    :[ 0.2],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[300],
                     "subsample"       :[1.0],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3]
        }

In [23]:
comb = compute_comb(params)
print(comb[0])
weights = [0.11, 0.32, 0.12, 0.1, 0.25, 0.1]
accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,comb[0],weights)

[0.2, 2, 300, 1.0, 0, 0.5, 1, 3]


In [24]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))

0.6678571428571428
0.6687028657616892
0.66
0.6643514328808446


In [95]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))
# parameters : [0.2, 2, 300, 1.0, 0, 0.5, 1, 3]
# weights = [0.11, 0.32, 0.12, 0.1, 0.25, 0.1]

0.6678571428571428
0.6687028657616892
0.66
0.6643514328808446


In [47]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))
# parameters : [0.2, 2, 300, 1.0, 0, 0.5, 1, 3]
# weights = [0.11, 0.32, 0.15, 0.1, 0.22, 0.1]

0.6678571428571428
0.6686651583710407
0.66
0.6643325791855204


In [43]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))
# parameters : [0.2, 2, 300, 1.0, 0, 0.5, 1, 3]
# weights = [0.1, 0.35, 0.15, 0.1, 0.2, 0.1]

0.6660714285714285
0.6686651583710407
0.64
0.6543325791855203


In [ ]:
parameters : [0.2, 2, 300, 1.0, 0, 0.5, 1, 3]
accuracy : 0.6678571428571428
spes     : 0.6725867269984918
sens     : 0.615
auc      : 0.6437933634992459
weights  : [0.1, 0.25, 0.2, 0.1, 0.2, 0.15]

In [ ]:
parameters : [0.3, 4, 100, 0.75, 0.5, 1.0, 2, 3]
accuracy : 0.6535714285714286
spes     : 0.6552036199095023
sens     : 0.64
auc      : 0.6476018099547511

In [149]:
# print(weight_scores[0])
k = weight_scores
sum_acc = 0
sum_sens = 0
sum_auc  =0
for i in range(6):
    acc = []
    sens = []
    auc = []
    for j in range(10):
        acc.append(weight_scores[i][j][0])
        sens.append(weight_scores[i][j][1])
        auc.append(weight_scores[i][j][2])
    acc = np.array(acc)
    sens= np.array(sens)
    auc = np.array(auc)
    print("accuracy :", acc.mean(), " sensitiv :", sens.mean(), " auc:", auc.mean())
    print(acc.mean()/3.5879545454545454,"   ", sens.mean()/2.3754007936507935, "    ", auc.mean()/3.482470238095238)
    print((acc.mean()*3.5879545454545454+ sens.mean()*2.3754007936507935+ auc.mean()*3.482470238095238)/9.445825577200576)
    print("--------------------------------------------------------------------------")
    sum_acc = sum_acc + acc.mean()
    sum_sens = sum_sens + sens.mean()
    sum_auc = sum_auc + auc.mean()
print("sum_acc ",sum_acc)
print("sum_sens",sum_sens)
print("sum_auc ",sum_auc)
print(sum_acc+sum_sens+sum_auc)

accuracy : 0.6067424242424242  sensitiv : 0.4366785714285714  auc: 0.6045386904761905
0.16910538206541245     0.18383363876772676      0.17359479023339686
0.5631629356276313
--------------------------------------------------------------------------
accuracy : 0.6399999999999999  sensitiv : 0.4618253968253968  auc: 0.6316964285714285
0.17837461202255017     0.19441998927499288      0.18139320234849712
0.5921319729584433
--------------------------------------------------------------------------
accuracy : 0.5639393939393939  sensitiv : 0.3386190476190476  auc: 0.5261309523809523
0.15717573530964293     0.14255238464352715      0.15107981300903334
0.4933375289673432
--------------------------------------------------------------------------
accuracy : 0.5574999999999999  sensitiv : 0.316531746031746  auc: 0.49117559523809523
0.1553810096915183     0.13325403733037533      0.1410422951688303
0.4724498431028997
--------------------------------------------------------------------------
accura

In [142]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))

0.625
0.6339743589743589
0.515
0.5744871794871795


In [26]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("learning_rate")
    list2 = params.get("max_depth")
    list3 = params.get("n_estimators") 
    list4 = params.get("min_child_weight")
    list5 = params.get("gamma")    
    list6 = params.get("colsample_bytree")
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")

#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n, o, p] for i in list1  
                                    for j in list2 
                                    for k in list3 
                                    for l in list4 
                                    for m in list5 
                                    for n in list6 
                                    for o in list7
                                    for p in list8] 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [27]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [28]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params,weights):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    combs = compute_comb(params)
    print(len(combs))
    for j in range(len(combs)):
        print(j)
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,combs[j],weights)
        if mean(sensitivity)> 0.5 and mean(specificity)>0.6 and mean(accuracy)>0.6 and mean(auc)>0.5:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print (style.BOLD + 'weights     ' + style.END, weights)
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
        else:
            print(combs[j])
            print("accuracy    ",mean(accuracy))
            print("specificity ",mean(specificity))
            print("sensitivity ",mean(sensitivity))
            print("auc         ",mean(auc))
            print("weights     ",weights)
            
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc), weights])
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params

In [ ]:
#     weights = [0.16, 0.173, 0.2, 0.147, 0.16, 0.16]
0.6589285714285714
0.6629713423831071
0.615
0.6389856711915536

# Ratio = 1

    parameters : [0.2, 3, 200, 1, 0, 0.5, 1, 1]
    accuracy : 0.5482142857142858
    spes     : 0.5339366515837104
    sens     : 0.735
    auc      : 0.6344683257918552


In [48]:
params = {
                    "learning_rate"    :[ 0.2],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[200],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[1]
        }

In [59]:
total= []
for i in range(5,35,5):
    ii = i/100
    for j in range(5,35,5):
        jj = j/100
        for k in range(5,35,5):
            kk = k/100
            for l in range(5,35,5):
                ll = l/100
                for m in range(5,35,5):
                    mm = m/100
                    for n in range(5,35,5):
                        nn = n/100
                        for o in range(5,35,5):
                            oo = o/100
                            for p in range(5,35,5):
                                pp = p/100
                                for q in range(5,35,5):
                                    qq = q/100
                                    for r in range(5,35,5):
                                        rr = r/100
                                        for s in range(5,35,5):
                                            ss = s/100
                                            for t in range(5,35,5):
                                                tt = t/100
                                                if ii+jj+kk+ll+mm+nn++oo+pp+qq+rr+ss+tt == 1:
                                                    weights = [ii, jj, kk, ll, mm, nn, oo, pp,qq, rr, ss, tt]
                                                    print(weights)
                                                    max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                                                    total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.2, 0.3]
1
0
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 12
1
76
2
76
3
76
4
76
5
76
6
76
7
76
8
76
9
76
10
76
11
76
12
75
numOfSubsamples 13
1
72
2
72
3
72
4
72
5
72
6
72
7
72
8
72
9
72
10
72
11
72
12
72
13
72


IndexError: list index out of range

In [15]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.57:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 4]
accuracy : 0.8589285714285715
spes     : 0.907579185520362
sens     : 0.245
auc      : 0.576289592760181
weights  : [0.1, 0.4, 0.1, 0.15, 0.1, 0.15]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 4]
accuracy : 0.8589285714285715
spes     : 0.907579185520362
sens     : 0.245
auc      : 0.576289592760181
weights  : [0.1, 0.4, 0.1, 0.15, 0.15, 0.1]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 4]
accuracy : 0.8553571428571428
spes     : 0.9037330316742082
sens     : 0.245
auc      : 0.5743665158371041
weights  : [0.1, 0.4, 0.15, 0.1, 0.1, 0.15]
------------------------------------------------
parameters : [0.1, 2, 100, 1, 0, 1.0, 1, 4]
accuracy : 0.8553571428571428
spes     : 0.9037330316742082
sens     : 0.245
auc      : 0.5743665158371041
weights  : [0.1, 0.4, 0.15, 0.1, 0.15, 0.1]
------------------------------------------------
parameters : [0.1, 2, 100, 1

## Δεν βλέπουμε βελτίωση 

    parameters : [0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
    accuracy : 0.6821428571428572
    spes     : 0.6879336349924585
    sens     : 0.615
    auc      : 0.6514668174962293


In [16]:
params = {
                    "learning_rate"    :[ 0.1],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[1.0], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4]
        }

In [ ]:
total= []
for i in range(10,45,5):
    ii = i/100
    for j in range(10,45,5):
        jj = j/100
        for k in range(10,45,5):
            kk = k/100
            for l in range(10,45,5):
                ll = l/100
                for m in range(10,45,5):
                    mm = m/100
                    for n in range(10,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
                            print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

[0.1, 0.1, 0.1, 0.1, 0.2, 0.4]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.1, 0.1, 0.2, 0.4]
[0.1, 0.1, 0.1, 0.1, 0.25, 0.35]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.1, 0.1, 0.25, 0.35]
[0.1, 0.1, 0.1, 0.1, 0.3, 0.3]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.1, 0.1, 0.3, 0.3]
[0.1, 0.1, 0.1, 0.1, 0.35, 0.25]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.1, 0.1, 0.35, 0.25]
[0.1, 0.1, 0.1, 0.1, 0.4, 0.2]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.1, 0.1, 0.4, 0.2]
[0.1, 0.1, 0.1, 0

[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.15, 0.15, 0.15, 0.35]
[0.1, 0.1, 0.15, 0.15, 0.2, 0.3]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.15, 0.15, 0.2, 0.3]
[0.1, 0.1, 0.15, 0.15, 0.25, 0.25]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.15, 0.15, 0.25, 0.25]
[0.1, 0.1, 0.15, 0.15, 0.3, 0.2]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.15, 0.15, 0.3, 0.2]
[0.1, 0.1, 0.15, 0.15, 0.35, 0.15]
1
0
[0.1, 2, 100, 1, 1, 1.0, 1.5, 4]
accuracy     0.9267857142857143
specificity  1.0
sensitivity  0.0
auc          0.5
weights      [0.1, 0.1, 0.15, 0.15, 0.35, 0.15]
[0.1, 0.1, 0.15, 0.15, 0.4, 0.1]